In [52]:
import nltk
import requests
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
NEWS_API_KEY = "60697fb4390d400b83ef934d1c2d0dc1"

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/aadi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [53]:
def fetch_news():
    url = f"https://newsapi.org/v2/everything?q=geopolitics&language=en&pageSize=10&sortBy=publishedAt&apiKey={NEWS_API_KEY}"
    response=requests.get(url)
    data=response.json()
    articles=data.get("articles",[])
    news = [article["title"] + ". " + str(article["description"]) for article in articles]
    return news


In [54]:


def analyze_sentiment(news_list):
    results=[]
    for news in news_list:
        sentiment=sia.polarity_scores(news)
        results.append({
            "news":news,
            "sentiment_score":sentiment["compound"]
        })
    return results

In [55]:
news=fetch_news()
sentiment_result=analyze_sentiment(news)

for item in sentiment_result:
    print(f"##{item['news']}\n  -> Sentiment Score: {item['sentiment_score']}\n")
    

##FO° Talks: The New Geopolitical Landscape of the Middle East Writeup. Atul Singh: Welcome to FO° Talks. With me is Gary Grappo. He’s the former chair of Fair Observer. He has been an ambassador for the US. He has had a glorious diplomatic career spanning many decades in many countries. He speaks many languages, and few people h…
  -> Sentiment Score: 0.8591

##BlackRock CEO Larry Fink’s grim recession warning: Trump tariffs ‘beyond anything I could have imagined’. Trump’s decision on April 2 to impose the most severe tariffs in over a century triggered a global sell-off.
  -> Sentiment Score: -0.9184

##Despite Zuckerberg’s Promises, Facebook ‘Fact Checkers’ Continue Censoring Information About COVID-19 Vaccines. In early January 2025 Meta announced that it would be scrapping the use of its so-called ‘independent fact checkers’ on Facebook and Instagram, replacing them with a ‘community notes’ system similar to that now used on Elon Musk’s X platform. …
The post Despi…
  -> Sentiment